In [62]:
import pandas as pd
import numpy as np
import mysql.connector

In [63]:
df_tabla_aerolineas = pd.read_csv("tabla_aerolineas.csv")
df_tabla_aeropuertos = pd.read_csv("tabla_aeropuertos.csv")
df_tabla_distancias = pd.read_csv("tabla_distancias.csv")
df_tabla_estados = pd.read_csv("tabla_estados.csv")
df_tabla_vuelos = pd.read_csv("tabla_vuelos.csv")

In [64]:
df_tabla_aerolineas['founded'] = pd.to_numeric(df_tabla_aerolineas['founded'], errors='coerce').astype('Int64')
df_tabla_aerolineas['started_operations'] = pd.to_numeric(df_tabla_aerolineas['started_operations'], errors='coerce').astype('Int64')

In [65]:
df_tabla_aerolineas = df_tabla_aerolineas.replace({np.nan: None, pd.NA: None})
df_tabla_aeropuertos = df_tabla_aeropuertos.replace({np.nan: None, pd.NA: None})
df_tabla_distancias = df_tabla_distancias.replace({np.nan: None, pd.NA: None})
df_tabla_estados = df_tabla_estados.replace({np.nan: None, pd.NA: None})
df_tabla_vuelos = df_tabla_vuelos.replace({np.nan: None, pd.NA: None})

In [66]:
df_tabla_estados = df_tabla_estados.drop_duplicates(subset='estado')

In [67]:
time_format_hh_mm_ss = '%H:%M:%S'
time_format_hh_mm = '%H:%M'

df_tabla_vuelos['hora_salida_programada'] = pd.to_datetime(
    df_tabla_vuelos['hora_salida_programada'], format=time_format_hh_mm_ss, errors='coerce'
).dt.time

df_tabla_vuelos['hora_salida_real'] = pd.to_datetime(
    df_tabla_vuelos['hora_salida_real'], format=time_format_hh_mm_ss, errors='coerce'
).dt.time

df_tabla_vuelos['hora_llegada_real'] = pd.to_datetime(
    df_tabla_vuelos['hora_llegada_real'], format=time_format_hh_mm_ss, errors='coerce'
).dt.time

df_tabla_vuelos['hora_despegue'] = pd.to_datetime(
    df_tabla_vuelos['hora_despegue'], format=time_format_hh_mm, errors='coerce'
).dt.time

In [68]:
df_tabla_aerolineas.astype(str).map(len).max()

ICAO                   3
aerolinea             18
IATA                   2
country               13
founded                4
started_operations     4
Group                 25
Base                  51
fleet_size             3
average_fleet_Age      4
official_site         20
dtype: int64

In [69]:
df_tabla_aerolineas.columns[df_tabla_aerolineas.isnull().any()].tolist()

['founded', 'started_operations', 'Group', 'Base']

In [70]:
create_aerolineas_table_query = """
CREATE TABLE IF NOT EXISTS aerolineas (
    ICAO VARCHAR(3) NOT NULL,
    aerolinea VARCHAR(20),
    IATA VARCHAR(2),
    country VARCHAR(30),
    founded YEAR NULL,
    started_operations YEAR NULL,
    `group` VARCHAR(40) NULL,
    base TINYTEXT NULL,
    fleet_size SMALLINT UNSIGNED,
    average_fleet_age FLOAT UNSIGNED,
    official_site VARCHAR(40),
    PRIMARY KEY (ICAO)
);
"""

In [71]:
df_tabla_aeropuertos.astype(str).map(len).max()

codigo_aeropuerto     3
nombre_aeropuerto    52
ciudad               26
estado                2
latitude             18
longitude            19
direccion            87
dtype: int64

In [72]:
df_tabla_aeropuertos.columns[df_tabla_aeropuertos.isnull().any()].tolist()

['direccion']

In [73]:
create_aeropuertos_table_query = """
CREATE TABLE IF NOT EXISTS aeropuertos (
    codigo_aeropuerto VARCHAR(3) NOT NULL,
    nombre_aeropuerto VARCHAR(100),
    ciudad VARCHAR(40),
    estado VARCHAR(2),
    latidude FLOAT,
    longitude FLOAT,
    direccion TINYTEXT NULL,
    PRIMARY KEY (codigo_aeropuerto),
    FOREIGN KEY (estado) REFERENCES estados (estado)
);
"""

In [74]:
df_tabla_distancias.astype(str).map(len).max()

distancia_millas      6
aeropuerto_origen     4
aeropuerto_destino    4
dtype: int64

In [75]:
df_tabla_distancias.columns[df_tabla_distancias.isnull().any()].tolist()

['aeropuerto_origen', 'aeropuerto_destino']

In [76]:
create_distancias_table_query = """
CREATE TABLE IF NOT EXISTS distancias (
    distancia_id INT AUTO_INCREMENT NOT NULL,
    distancia_millas FLOAT,
    aeropuerto_origen VARCHAR(3) NULL,
    aeropuerto_destino VARCHAR(3) NULL,
    PRIMARY KEY (distancia_id),
    FOREIGN KEY (aeropuerto_origen) REFERENCES aeropuertos (codigo_aeropuerto),
    FOREIGN KEY (aeropuerto_destino) REFERENCES aeropuertos (codigo_aeropuerto)
)
"""

In [77]:
df_tabla_estados.astype(str).map(len).max()

estado            2
nombre_estado    33
dtype: int64

In [78]:
df_tabla_estados.columns[df_tabla_estados.isnull().any()].tolist()

[]

In [79]:
create_estados_table_query = """
CREATE TABLE IF NOT EXISTS estados (
    estado VARCHAR(2) NOT NULL,
    nombre_estado VARCHAR(50),
    PRIMARY KEY (estado)
)
"""

In [80]:
df_tabla_vuelos.columns[df_tabla_vuelos.isnull().any()].tolist()

['hora_despegue']

In [81]:
df_tabla_vuelos.describe().loc[['min', 'max']]

,numero_vuelo,duracion_programada_vuelo,duracion_real,retraso_salida,tiempo_pista_salida,tiempo_retraso_aerolinea,tiempo_retraso_clima,tiempo_retraso_sistema_aviacion,tiempo_retraso_seguridad,retraso_llegada,festivos
min,1.0,-56.0,0.0,-60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,9679.0,685.0,736.0,3001.0,218.0,2976.0,1392.0,1147.0,983.0,1615.0,1.0


In [82]:
df_tabla_vuelos['festivos'].unique()

array([0, 1])

In [83]:
create_vuelos_table_query = """
CREATE TABLE IF NOT EXISTS vuelos (
    vuelo_id INT AUTO_INCREMENT NOT NULL,
    aerolinea VARCHAR(3),
    fecha DATETIME,
    numero_vuelo SMALLINT UNSIGNED,
    numero_cola VARCHAR(6),
    hora_salida_programada TIME,
    hora_salida_real TIME,
    duracion_programada_vuelo SMALLINT,
    duracion_real SMALLINT UNSIGNED,
    retraso_salida SMALLINT,
    hora_despegue TIME NULL,
    tiempo_pista_salida SMALLINT UNSIGNED,
    tiempo_retraso_aerolinea SMALLINT UNSIGNED,
    tiempo_retraso_clima SMALLINT UNSIGNED,
    tiempo_retraso_sistema_aviacion SMALLINT UNSIGNED,
    tiempo_retraso_seguridad SMALLINT UNSIGNED,
    retraso_llegada SMALLINT UNSIGNED,
    aeropuerto_origen VARCHAR(3),
    hora_llegada_real TIME,
    festivos TINYINT(1) NOT NULL DEFAULT 0,
    aeropuerto_destino VARCHAR(3),
    PRIMARY KEY (vuelo_id),
    FOREIGN KEY (aerolinea) REFERENCES aerolineas (ICAO),
    FOREIGN KEY (aeropuerto_origen) REFERENCES aeropuertos (codigo_aeropuerto),
    FOREIGN KEY (aeropuerto_destino) REFERENCES aeropuertos (codigo_aeropuerto)
);
"""

In [84]:
create_tables_queries = [
    create_aerolineas_table_query,
    create_estados_table_query,
    create_aeropuertos_table_query,
    create_distancias_table_query,
    create_vuelos_table_query
]

In [110]:
database = "airlines_database"
db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = None)

cursor = db.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
cursor.execute("USE airlines")

In [86]:
for query in create_tables_queries:
    cursor.execute(query)
db.commit()

In [87]:
insert_aerolineas_query = """
INSERT INTO aerolineas (ICAO, aerolinea, IATA, country, founded, started_operations, `group`, base, fleet_size, average_fleet_age, official_site)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [ ]:
data_to_insert_aerolineas = [tuple(x) for x in df_tabla_aerolineas.values]
cursor.executemany(insert_aerolineas_query, data_to_insert_aerolineas)
db.commit()

In [91]:
insert_estados_query = """
INSERT INTO estados (estado, nombre_estado)
VALUES (%s, %s);
"""

In [92]:
data_to_insert_estados = [tuple(x) for x in df_tabla_estados.values]
cursor.executemany(insert_estados_query, data_to_insert_estados)
db.commit()

In [93]:
insert_aeropuertos_query = """
INSERT INTO aeropuertos (codigo_aeropuerto, nombre_aeropuerto, ciudad, estado, latidude, longitude, direccion)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

In [94]:
data_to_insert_aeropuertos = [tuple(x) for x in df_tabla_aeropuertos.values]
cursor.executemany(insert_aeropuertos_query, data_to_insert_aeropuertos)
db.commit()

In [95]:
insert_distancias_query = """
INSERT INTO distancias (distancia_millas, aeropuerto_origen, aeropuerto_destino)
VALUES (%s, %s, %s);
"""

In [96]:
data_to_insert_distancias = [tuple(x) for x in df_tabla_distancias.values]
cursor.executemany(insert_distancias_query, data_to_insert_distancias)
db.commit()

In [106]:
insert_vuelos_query = """
INSERT INTO vuelos (aerolinea, fecha, numero_vuelo, numero_cola, hora_salida_programada, hora_salida_real, duracion_programada_vuelo, duracion_real, retraso_salida, hora_despegue, tiempo_pista_salida, tiempo_retraso_aerolinea, tiempo_retraso_clima, tiempo_retraso_sistema_aviacion, tiempo_retraso_seguridad, retraso_llegada, aeropuerto_origen, hora_llegada_real, festivos, aeropuerto_destino)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [111]:
batch_size = 1000
data_to_insert_vuelos = [tuple(x) for x in df_tabla_vuelos.values]

for i in range(0, len(data_to_insert_vuelos), batch_size):
    batch = data_to_insert_vuelos[i:i + batch_size]
    cursor.executemany(insert_vuelos_query, batch)
    db.commit()

In [112]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

for (table_name,) in tables:
    print(f"Fetching data from table: {table_name}")
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
    rows = cursor.fetchall()
    print(f"Table: {table_name}")
    for row in rows:
        print(row)
    print("\n")

Fetching data from table: aerolineas
Table: aerolineas
('AAL', 'American Airlines', 'AA', 'United States', 1930, None, 'American Airlines Group', 'Dallas / Fort Worth International (DFW / KDFW)', 967, 13.3, 'aa.com')
('AAY', 'Allegiant Air', 'G4', 'United States', 1997, 1998, None, 'Las Vegas Harry Reid International (LAS / KLAS)', 126, 15.6, 'allegiantair.com')
('ASA', 'Alaska Airlines', 'AS', 'United States', 1932, 1944, 'Alaska Air Group', 'Seattle Tacoma International (SEA / KSEA)', 317, 8.8, 'alaskaair.com')
('ASH', 'Mesa Airlines', 'YV', 'United States', 1980, 1980, 'Mesa Air Group', 'Phoenix Sky Harbor (PHX / KPHX)', 125, 12.1, 'mesa-air.com')
('DAL', 'Delta Air Lines', 'DL', 'United States', 1928, 1929, None, 'Atlanta Hartsfield - Jackson (ATL / KATL)', 984, 15.3, 'delta.com')
('EDV', 'Endeavor Air', '9E', 'United States', None, 2013, 'Delta Air Lines', 'Minneapolis St. Paul International (MSP / KMSP)', 142, 14.4, 'endeavorair.com')
('ENY', 'Envoy Air', 'MQ', 'United States', N